![Erudio logo](img/erudio-logo-small.png)

# Functions and Classes

The fundamental units of modularity in Python are packages, modules, classes, and functions.  Creating packages and modules is outside the scope of this introductory course, but every time you utilize `import foo` or `from foo.bar import baz as bz` you are utilizing those means of packaging related functionality.

What you *will* write a lot of, even as a beginner or intermediate Python user, is **functions**.  Classes, in turn, are essentially just ways of bundling together several functions, and often some data those functions use, into the same Python object.

## Functions

A function is simply a mechanism to encapsulate a number of program steps that you could, in concept, simply write sequentially in a main program.  Most of the time, functions also take a collection of parameters.  These parameters can either be *positional* or *named*.

Continuing the conceit of the last lesson, let's write a few functions to create ASCII art versions of the Mandelbrot set.  Recall that the Mandelbrot set is defined  in the complex plane as the complex numbers c for which the function $f_{c}(z)=z^{2}+c$ does not diverge to infinity when iterated starting at $z=0$. There is a nice property of this iteration that if any iteration of the function has an absolute value greater than 2, we know already that the function (indexed by that complex number) diverges.

That may be a mouthful, and the code can make it clearer.  The first step in our approach will be to create a grid of points representing in iteration of the Mandelbrot generating function across a region of the complex plane.  We will use a `list` of `namedtuple` objects in a simlar way as we did with the earlier ASCII art.

### Create a base data structure

In [ ]:
from collections import namedtuple
from typing import List

Point = namedtuple("Point", ["c", "z", "iteration"])

def make_comlex_grid(  # type hints do not change runtime behavior
    x_min: float = -2, 
    x_max: float = 0.5, 
    y_min: float = -1, 
    y_max: float = 1,
    x_ticks: int = 80,
    y_ticks: int = 30,
) -> List[Point]:
    # Arrange the points left-to-right, top-to-bottom
    y_step = (y_max - y_min) / y_ticks
    x_step = (x_max - x_min) / x_ticks

    points = []
    for n_imag in range(y_ticks):
        row = []
        for n_real in range(x_ticks):
            c = (x_min + (x_step * n_real)) + 1j * (y_max - (y_step * n_imag))
            row.append(Point(c, 0, 0))
        points.append(row)
    return points

In [ ]:
from pprint import pprint

grid = make_comlex_grid()
# Display a few points from top-left corner of the grid
pprint([row[:3] for row in grid[:3]])

At this point, we simply have a grid of 60×30 points, each at iteration zero.  

### Implement "orbits" of the Mandelbrot function

We can iterate each of these points individually, since the so-called *orbit* of each is independent of the others.  Let's write a function for that.  

If any point diverges, we will simply return its value at the iteration where it diverged rather than continue to calculate values.

In [ ]:
def iterate_point(point: Point) -> Point:
    if abs(point.z) > 2:  # already diverged
        return point
    return Point(point.c, point.z**2 + point.c, point.iteration + 1)

As an example, let's loop through a few iterations of a particular point in the grid.  There's a small hack below because iterations can be *too fast* for the Jupyter server to keep up if we pick a point with many orbits (we wait a microsecond artificially between iterations).

In [ ]:
from time import sleep

p = grid[14][15]
while abs(p.z) < 2:
    print(p)
    sleep(1e-6)
    p = iterate_point(p)

We see how iteration and "escape" happens for one point, but let's write a function to evolve the entire grid of complex poionts.

In [ ]:
from copy import deepcopy

def evolve_grid(grid, n_steps=100):
    new_grid = deepcopy(grid)  # Leave the original grid as it was
    for _ in range(n_steps):
        for y_dim, row in enumerate(new_grid):
            for x_dim, point in enumerate(row):
                new_grid[y_dim][x_dim] = iterate_point(point)
    return new_grid

In [ ]:
new_grid = evolve_grid(grid, 120)

# The top-left corner escapes immediately, let's look somewhere in the middle
pprint([row[12:15] for row in new_grid[15:18]])

As most students will have seen, sometimes visually appealing displays of the Mandelbrot set use the iteration where the orbits escape to represent different colors, often producing striking patterns.  In the few points shown, some escape their orbit quickly, and others either never will, or at least will take more than the 120 steps that we ran.

### Visualization

To round out this example, let's create an ASCII art visualization of the grid in the complex plane we have already implemented.

In [ ]:
def show_grid(grid, fillchar="•"):
    ascii = ""
    for row in grid:
        for point in row:
            ascii += "".join(fillchar if abs(point.z) <= 2 else " ")
        ascii += "\n"            
    return ascii

In [ ]:
from IPython.display import HTML

image = show_grid(evolve_grid(make_comlex_grid()))
HTML(f"<pre>{image}</pre>")

## Classes

The several functions written here all operate on the same—or at least closely related—data.  How you would write this class is left as an **exercise** for students to work on after this session is over.  The key thing to understand about class is simply that they often refer to their own instances.  As well, a number of "magic methods" of classes can affect the behavior of their instances.

Let's implement a small class, mostly only to show its syntax and a few *magic methods* (their pattern of beginning and ending with double underscores makes them often informally named as "dunders").

In [ ]:
from string import ascii_lowercase
from random import shuffle

class ASCIIArt:
    def __init__(self, x_dim, y_dim):
        self.grid = []
        for _ in range(y_dim):
            letters = list(ascii_lowercase + " "*6)
            shuffle(letters)
            self.grid.append(letters[:x_dim])
            
    def reshuffle(self):
        for n, row in enumerate(self.grid):
            shuffle(row)
            self.grid[n] = row
        shuffle(self.grid)

    def __str__(self):
        return "\n".join("".join(row) for row in self.grid)        

In [ ]:
my_art = ASCIIArt(15, y_dim=6)
print(my_art)

In [ ]:
my_art.reshuffle()
print(my_art)

## Exercise

Create a `Mandelbrot` class that implements the functionality of the several functions implemented in this lesson, and that provides an API you find helpful.

-----

Materials licensed under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/) by the authors